In [ ]:
# Fragestellung: Welche Spiele spielen Spieler, die das Spiel X mehr als 50h (3000 min) gespielt haben?
# text    text      list[text]
# item_id item_name item_ids

# Kann ich speichern???


In [1]:
from cassandra.cluster import Cluster
print("Connecting...")
# Jede Node des Clusters ist gleich, daher können ebenfalls "cassandra1" und "cassandra2" genutzt werden
cluster = Cluster(['cassandra0']) 
session = cluster.connect()
print("Connection established!")

Connecting...
Connection established!


In [19]:
create_keyspace = "CREATE KEYSPACE games WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor':2}"
res = session.execute(create_keyspace)
res

AlreadyExists: Keyspace 'games' already exists

In [8]:
use_keyspace = "USE games"
res = session.execute(use_keyspace)
res

In [98]:
create_table = """CREATE TABLE related_games_by_game_id (
 game_id text,
 game_name text,
 related_games list<text>,
 PRIMARY KEY(game_id, game_name)
)
;"""
res = session.execute(create_table)
res

In [65]:
# NICHT SO MACHEN!!!
# Weiter unten ist ein Beispiel wie man richtig Batch inserts macht.

items = [("a","100"),("b","1"),("f","3"),("e","2"),]
for game_id, game_name in items:
    insert = f"""
    INSERT INTO related_games_by_game_id (game_id, game_name, related_games)
           VALUES ('{game_id}', '{game_name}', ['other_game_id_1','other_game_id_2','other_game_id_3']);
    """
    res = session.execute(insert)
    print(res)

In [101]:
select = "SELECT * FROM related_games_by_game_id"
rows = session.execute(select)
for row in rows:
  print(row)

Row(game_id='a', game_name='100', related_games=['other_game_id_1', 'other_game_id_2', 'other_game_id_3'])
Row(game_id='f', game_name='3', related_games=['other_game_id_1', 'other_game_id_2', 'other_game_id_3'])
Row(game_id='e', game_name='2', related_games=['other_game_id_1', 'other_game_id_2', 'other_game_id_3'])
Row(game_id='b', game_name='1', related_games=['other_game_id_1', 'other_game_id_2', 'other_game_id_3'])


In [97]:
select = "DROP TABLE related_games_by_game_id"
res = session.execute(select)
res

InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'games.related_games_by_game_id' doesn't exist"

In [99]:
from cassandra.query import BatchStatement, ConsistencyLevel

# Achtung! Hier wird ein Antipattern verwendet
# Die game_id ist der partition key. Bei diesem Datensatz ist er bei jedem Eintrag unterschiedlich.
# Dadurch muss bei jedem insert unter Umständen mit einer anderen Node gesprochen werden, wodurch die Operation langsam wird.
# Batch inserts sollten so organisiert werden, dass Zeilen mit gleichem partition key zusammen ausgeführt werden.
# https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useBatchBadExample.html

items = [("a","100"),("b","1"),("f","3"),("e","2")]

insert_game = session.prepare("INSERT INTO related_games_by_game_id (game_id, game_name, related_games) VALUES (?, ?, ['other_game_id_1','other_game_id_2','other_game_id_3'])")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for game_id, game_name in items:
    batch.add(insert_game, (game_id, game_name))

res = session.execute(batch)